# 4교시 조인 연산

### 목차
* [1. 조인 유형](#1.-조인-유형)
* [2. Inner Join](#2.-Inner-Join)
* [3. Outer Join](#3.-Outer-Join)
* [4. 조인 유의사항](#4.-조인-유의사항)
* [참고자료](#참고자료)

### 1. 조인 유형

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)
# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

### JOIN 학습을 위해 상품은 단 하나만 구매할 수 있다고 가정하여 아래와 같은 테이블이 존재합니다
#### 정보 1. 고객은 4명이지만, 1명은 탈퇴하여 존재하지 않습니다
| 고객 아이디 (u_id) | 고객 이름 (u_name) | 고객 성별 (u_gender) |
| - | - | - |
| 1 | 정휘센 | 남 |
| 2 | 김싸이언 | 남 |
| 3 | 박트롬 | 여 |

#### 정보 2. 구매 상품은 3개이며, 탈퇴한 고객의 상품정보가 남아있습니다
| 구매 고객 아이디 (u_id) | 구매 상품 이름 (p_name) | 구매 상품 가격 (p_amount) |
| - | - | - |
| 2 | LG DIOS | 2,000,000 |
| 3 | LG Cyon | 1,800,000 |
| 4 | LG Computer | 4,500,000 |


In [3]:
user = spark.createDataFrame([
    (1, "정휘센", "남"),
    (2, "김싸이언", "남"),
    (3, "박트롬", "여")
]).toDF("u_id", "u_name", "u_gender")
user.printSchema()
display(user)
    
purchase = spark.createDataFrame([
    (2, "LG DIOS", 2000000),
    (3, "LG Cyon", 1800000),
    (4, "LG Computer", 4500000)
]).toDF("p_uid", "p_name", "p_amont")
purchase.printSchema()
display(purchase)

root
 |-- u_id: long (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)



u_id,u_name,u_gender
1,정휘센,남
2,김싸이언,남
3,박트롬,여


root
 |-- p_uid: long (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amont: long (nullable = true)



p_uid,p_name,p_amont
2,LG DIOS,2000000
3,LG Cyon,1800000
4,LG Computer,4500000


## 2. Inner Join
### 2.1 구매 정보와 일치하는 고객 정보를 조인 (inner)

In [4]:
user.join(purchase, user.u_id == purchase.p_uid).show()
user.join(purchase, user.u_id == purchase.p_uid, "inner").count()

+----+--------+--------+-----+-------+-------+
|u_id|  u_name|u_gender|p_uid| p_name|p_amont|
+----+--------+--------+-----+-------+-------+
|   3|  박트롬|      여|    3|LG Cyon|1800000|
|   2|김싸이언|      남|    2|LG DIOS|2000000|
+----+--------+--------+-----+-------+-------+



2

### <font color=green>1. [기본]</font> 고객 정보 "data/tbl_user", 제품 정보 "data/tbl_purchase" CSV 파일을 읽고
#### 1. 각각 스키마를 출력하세요
#### 2. 각각 데이터를 출력하세요
#### 3. 고객(tbl_user) 테이블의 u_id 와 제품(tbl_purchase) 테이블의 p_uid 는 고객아이디 입니다
#### 4. 고객 테이블을 기준으로 어떤 제품을 구매하였는지 inner join 을 통해 조인해 주세요
#### 5. 조인된 최종 테이블의 스키마와 데이터를 출력해 주세요

<details><summary>[실습1] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
left = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/tbl_user.csv")
)
left.printSchema()
left.show()

right = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/tbl_purchase.csv")
)
right.printSchema()
right.show()

join_codition = left.u_id == right.p_uid
answer = left.join(right, join_codition, "inner")
answer.printSchema()
display(answer)
```

</details>


In [28]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)

+----+----------+--------+--------+
|u_id|    u_name|u_gender|u_signup|
+----+----------+--------+--------+
|   1|    정휘센|      남|19700808|
|   2|  김싸이언|      남|19710201|
|   3|    박트롬|      여|19951030|
|   4|    청소기|      남|19770329|
|   5|유코드제로|      여|20021029|
|   6|  윤디오스|      남|20040101|
|   7|  임모바일|      남|20040807|
|   8|  조노트북|      여|20161201|
|   9|  최컴퓨터|      남|20201124|
+----+----------+--------+--------+

root
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)

+----------+-----+----+-----------+--------+
|    p_time|p_uid|p_id|     p_name|p_amount|
+----------+-----+----+-----------+--------+
|1603651550|    0|1000|GoldStar TV|  100000|
|1603651550|    1|2000|    LG DIO

u_id,u_name,u_gender,u_signup,p_time,p_uid,p_id,p_name,p_amount
1,정휘센,남,19700808,1603651550,1,2000,LG DIOS,2000000
1,정휘센,남,19700808,1603694755,1,2000,LG Gram,1800000
2,김싸이언,남,19710201,1603673500,2,2001,LG Cyon,1400000
3,박트롬,여,19951030,1603652155,3,2002,LG TV,1000000
4,청소기,남,19770329,1603674500,4,2003,LG Computer,4500000
5,유코드제로,여,20021029,1603665955,5,2004,LG Gram,3500000
5,유코드제로,여,20021029,1603666155,5,2004,LG TV,2500000


## 3. Outer Join
### 3.1 모든 고객의 정보 구매 정보를 조인 (left_outer)

In [5]:
user.join(purchase, user.u_id == purchase.p_uid, "left_outer").orderBy(purchase.p_uid.asc()).show()

+----+--------+--------+-----+-------+-------+
|u_id|  u_name|u_gender|p_uid| p_name|p_amont|
+----+--------+--------+-----+-------+-------+
|   1|  정휘센|      남| null|   null|   null|
|   2|김싸이언|      남|    2|LG DIOS|2000000|
|   3|  박트롬|      여|    3|LG Cyon|1800000|
+----+--------+--------+-----+-------+-------+



### 3-2. 모든 상품에 대한 고객 정보를 조인 (right_outer)

In [6]:
user.join(purchase, user.u_id == purchase.p_uid, "right_outer").orderBy(purchase.p_uid.asc()).show()

+----+--------+--------+-----+-----------+-------+
|u_id|  u_name|u_gender|p_uid|     p_name|p_amont|
+----+--------+--------+-----+-----------+-------+
|   2|김싸이언|      남|    2|    LG DIOS|2000000|
|   3|  박트롬|      여|    3|    LG Cyon|1800000|
|null|    null|    null|    4|LG Computer|4500000|
+----+--------+--------+-----+-----------+-------+



### 3-3. 모든 고객과 상품에 대한 정보를 조인 (full_outer)

In [7]:
user.join(purchase, user.u_id == purchase.p_uid, "full_outer").orderBy(purchase.p_uid.asc()).show()

+----+--------+--------+-----+-----------+-------+
|u_id|  u_name|u_gender|p_uid|     p_name|p_amont|
+----+--------+--------+-----+-----------+-------+
|   1|  정휘센|      남| null|       null|   null|
|   2|김싸이언|      남|    2|    LG DIOS|2000000|
|   3|  박트롬|      여|    3|    LG Cyon|1800000|
|null|    null|    null|    4|LG Computer|4500000|
+----+--------+--------+-----+-----------+-------+



### <font color=green>2. [기본]</font> 고객 정보 "data/tbl_user", 제품 정보 "data/tbl_purchase" CSV 파일을 읽고
#### 1. 각각 스키마를 출력하세요
#### 2. 각각 데이터를 출력하세요
#### 3. 고객(tbl_user) 테이블의 u_id 와 제품(tbl_purchase) 테이블의 p_uid 는 고객아이디 입니다
#### 4. 모든 상품을 기준으로 구매하 고객정보를 조인해 주세요 (left: purchase, right: user, join: left_outer)
#### 5. 조인된 최종 테이블의 스키마와 데이터를 출력해 주세요
#### 6. 출력시에 상품 가격의 내림차순으로 정렬해 주세요

<details><summary>[실습2] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
left = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/tbl_purchase.csv")
)
left.printSchema()
# left.show()

right = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/tbl_user.csv")
)
right.printSchema()
# right.show()

join_codition = left.p_uid == right.u_id
answer = left.join(right, join_codition, "left_outer")
answer.printSchema()
display(answer.orderBy(desc("p_amount")))
```

</details>


In [25]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


root
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)

root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)

root
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)



p_time,p_uid,p_id,p_name,p_amount,u_id,u_name,u_gender,u_signup
1603674500,4,2003,LG Computer,4500000,4,청소기,남,19770329
1603665955,5,2004,LG Gram,3500000,5,유코드제로,여,20021029
1603666155,5,2004,LG TV,2500000,5,유코드제로,여,20021029
1603651550,1,2000,LG DIOS,2000000,1,정휘센,남,19700808
1603694755,1,2000,LG Gram,1800000,1,정휘센,남,19700808
1603673500,2,2001,LG Cyon,1400000,2,김싸이언,남,19710201
1603652155,3,2002,LG TV,1000000,3,박트롬,여,19951030
1603651550,0,1000,GoldStar TV,100000,null,null,null,null


### <font color=blue>3. [중급]</font> 고객 정보 "data/tbl_user", 제품 정보 "data/tbl_purchase" CSV 파일을 읽고
#### 1. 각각 스키마를 출력하세요
#### 2. 각각 데이터를 출력하세요
#### 3. 고객(tbl_user) 테이블의 u_id 와 제품(tbl_purchase) 테이블의 p_uid 는 고객아이디 입니다
#### 4. 모든 고객을 기준으로 구매한 상품 정보를 조인해 주세요 (left: user, right: purchase, join: left_outer)
#### 5. 조인된 최종 테이블의 스키마와 데이터를 출력해 주세요
#### 6. 출력시에 상품 가격(tbl_purchase.p_amount)의 내림차순으로 정렬해 주세요
#### 7. 상품가격이 없는 경우에는 등록일자(tbl_user.u_signup)가 최신으로 정렬해 주세요

<details><summary>[실습3] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
left = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/tbl_user.csv")
)
left.printSchema()
# left.show()

right = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/tbl_purchase.csv")
)
right.printSchema()
# right.show()

join_codition = left.u_id == right.p_uid
answer = left.join(right, join_codition, "left_outer")
answer.printSchema()
display(answer.orderBy(desc("p_amount"), desc("u_signup")))
```

</details>


In [24]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)

root
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)

root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)



u_id,u_name,u_gender,u_signup,p_time,p_uid,p_id,p_name,p_amount
4,청소기,남,19770329,1603674500,4,2003,LG Computer,4500000
5,유코드제로,여,20021029,1603665955,5,2004,LG Gram,3500000
5,유코드제로,여,20021029,1603666155,5,2004,LG TV,2500000
1,정휘센,남,19700808,1603651550,1,2000,LG DIOS,2000000
1,정휘센,남,19700808,1603694755,1,2000,LG Gram,1800000
2,김싸이언,남,19710201,1603673500,2,2001,LG Cyon,1400000
3,박트롬,여,19951030,1603652155,3,2002,LG TV,1000000
9,최컴퓨터,남,20201124,null,null,null,null,null
8,조노트북,여,20161201,null,null,null,null,null
7,임모바일,남,20040807,null,null,null,null,null


### <font color=red>4. [고급]</font> 고객 정보 "data/tbl_user", 제품 정보 "data/tbl_purchase" CSV 파일을 읽고
#### 1. 각각 스키마를 출력하세요
#### 2. 각각 데이터를 출력하세요
#### 3. 고객(tbl_user) 테이블의 u_id 와 제품(tbl_purchase) 테이블의 p_uid 는 고객아이디 입니다
#### 4. 모든 고객을 기준으로 모든 상품 정보를 조인해 주세요 (left: user, right: purchase, join: inner)
#### 5. 조인된 최종 테이블의 스키마와 데이터를 출력해 주세요
#### 6. 출력시에 상품 가격(tbl_purchase.p_amount)의 내림차순으로 정렬해 주세요
#### 7. 상품가격이 없는 경우에는 등록일자(tbl_user.u_signup)가 최신으로 정렬해 주세요

<details><summary>[실습4] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
left = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/tbl_user.csv")
)
left.printSchema()
# left.show()

right = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/tbl_purchase.csv")
)
right.printSchema()
# right.show()

join_codition = left.u_id == right.p_uid
answer = left.join(right, join_codition, "inner")
answer.printSchema()
display(answer.orderBy(desc("p_amount"), desc("u_signup")))
```

</details>


In [23]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)

root
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)

root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)



u_id,u_name,u_gender,u_signup,p_time,p_uid,p_id,p_name,p_amount
4,청소기,남,19770329,1603674500,4,2003,LG Computer,4500000
5,유코드제로,여,20021029,1603665955,5,2004,LG Gram,3500000
5,유코드제로,여,20021029,1603666155,5,2004,LG TV,2500000
1,정휘센,남,19700808,1603651550,1,2000,LG DIOS,2000000
1,정휘센,남,19700808,1603694755,1,2000,LG Gram,1800000
2,김싸이언,남,19710201,1603673500,2,2001,LG Cyon,1400000
3,박트롬,여,19951030,1603652155,3,2002,LG TV,1000000


### <font color=blue>5. [중급]</font> 실습 5. 고객 정보 "data/tbl_user", 제품 정보 "data/tbl_purchase" CSV 파일을 읽고
#### 1. 각각 스키마를 출력하세요
#### 2. 각각 데이터를 출력하세요
#### 3. 고객(tbl_user) 테이블의 u_id 와 제품(tbl_purchase) 테이블의 p_uid 는 고객아이디 입니다
#### 4. 모든 고객과 모든 상품 정보를 조인해 주세요 (left: user, right: purchase, join: full_outer)
#### 5. 조인된 최종 테이블의 스키마와 데이터를 출력해 주세요
#### 6. 출력시에 상품 가격(tbl_purchase.p_amount)의 내림차순으로 정렬해 주세요
#### 7. 상품가격이 없는 경우에는 등록일자(tbl_user.u_signup)가 최신으로 정렬해 주세요

<details><summary>[실습5] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
left = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/tbl_user.csv")
)
left.printSchema()
# left.show()

right = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/tbl_purchase.csv")
)
right.printSchema()
# right.show()

join_codition = left.u_id == right.p_uid
answer = left.join(right, join_codition, "full_outer")
answer.printSchema()
display(answer.orderBy(desc("p_amount"), desc("u_signup")))
```

</details>


In [22]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)

root
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)

root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)



u_id,u_name,u_gender,u_signup,p_time,p_uid,p_id,p_name,p_amount
4,청소기,남,19770329,1603674500,4,2003,LG Computer,4500000
5,유코드제로,여,20021029,1603665955,5,2004,LG Gram,3500000
5,유코드제로,여,20021029,1603666155,5,2004,LG TV,2500000
1,정휘센,남,19700808,1603651550,1,2000,LG DIOS,2000000
1,정휘센,남,19700808,1603694755,1,2000,LG Gram,1800000
2,김싸이언,남,19710201,1603673500,2,2001,LG Cyon,1400000
3,박트롬,여,19951030,1603652155,3,2002,LG TV,1000000
null,null,null,null,1603651550,0,1000,GoldStar TV,100000
9,최컴퓨터,남,20201124,null,null,null,null,null
8,조노트북,여,20161201,null,null,null,null,null


### <font color=green>6. [기본]</font> 아래의 조인 연산 결과에서 null 값에 대한 치환을 해주세요
#### 1. 고객아이디(u_id)와 고객이름(u_name), 성별(u_gender), 가입일자(u_signup) 기본값을 채워주세요
##### u_id = 0, u_name = '미확인', u_gender = '미확인', u_signup = '19700101'

<details><summary>[실습6] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
left = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/tbl_purchase.csv")
)
left.printSchema()
# left.show()

right = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/tbl_user.csv")
)
right.printSchema()
# right.show()

join_codition = left.p_uid == right.u_id
user_fill = { "u_id":0, "u_name":"미확인", "u_gender":"미확인", "u_signup":"19700101" }
answer = left.join(right, join_codition, "left_outer").na.fill(user_fill)
answer.printSchema()
display(answer.orderBy(asc("u_signup")))
```

</details>


In [6]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
left = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/tbl_user.csv")
)
right = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/tbl_purchase.csv")
)
join_codition = left.u_id == right.p_uid
answer = left.join(right, join_codition, "full_outer")
answer.printSchema()
display(answer)

root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)
 |-- p_time: integer (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)



u_id,u_name,u_gender,u_signup,p_time,p_uid,p_id,p_name,p_amount
1,정휘센,남,19700808,1603651550,1,2000,LG DIOS,2000000
1,정휘센,남,19700808,1603694755,1,2001,LG Gram,1800000
6,윤디오스,남,20040101,null,null,null,null,null
3,박트롬,여,19951030,1603652155,3,2003,LG TV,1000000
5,유코드제로,여,20021029,1603665955,5,2001,LG Gram,3500000
5,유코드제로,여,20021029,1603666155,5,2003,LG TV,2500000
9,최컴퓨터,남,20201124,null,null,null,null,null
4,청소기,남,19770329,1603674500,4,2004,LG Computer,4500000
8,조노트북,여,20161201,null,null,null,null,null
7,임모바일,남,20040807,null,null,null,null,null


## 4. 조인시 유의할 점


In [8]:
u = spark.createDataFrame([
    (1, "정휘센", "남"),
    (2, "김싸이언", "남"),
    (3, "박트롬", "여")
]).toDF("id", "name", "gender")
u.printSchema()
u.show()
    
p = spark.createDataFrame([
    (2, "LG DIOS", 2000000),
    (3, "LG Cyon", 1800000),
    (4, "LG Computer", 4500000)
]).toDF("id", "name", "amount")
p.printSchema()
p.show()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)

+---+--------+------+
| id|    name|gender|
+---+--------+------+
|  1|  정휘센|    남|
|  2|김싸이언|    남|
|  3|  박트롬|    여|
+---+--------+------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- amount: long (nullable = true)

+---+-----------+-------+
| id|       name| amount|
+---+-----------+-------+
|  2|    LG DIOS|2000000|
|  3|    LG Cyon|1800000|
|  4|LG Computer|4500000|
+---+-----------+-------+



### 3.1 중복 컬럼명 처리가 되지 않은 경우
> #### AnalysisException: "Reference 'id' is ambiguous, could be: id, id.;"

In [9]:
up = u.join(p, u.id == p.id)
up.show()
# up.select("id")

+---+--------+------+---+-------+-------+
| id|    name|gender| id|   name| amount|
+---+--------+------+---+-------+-------+
|  3|  박트롬|    여|  3|LG Cyon|1800000|
|  2|김싸이언|    남|  2|LG DIOS|2000000|
+---+--------+------+---+-------+-------+



### 3.2 중복 컬럼명 해결방안 - 데이터 프레임의 컬럼 명을 다르게 만든다

In [10]:
u1 = u.withColumnRenamed("id", "u_uid")
p1 = p.withColumnRenamed("id", "p_uid")
u1.printSchema()
p1.printSchema()

up = u1.join(p1, u1.u_uid == p1.p_uid)
up.show()
up.select("u_uid")

root
 |-- u_uid: long (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)

root
 |-- p_uid: long (nullable = true)
 |-- name: string (nullable = true)
 |-- amount: long (nullable = true)

+-----+--------+------+-----+-------+-------+
|u_uid|    name|gender|p_uid|   name| amount|
+-----+--------+------+-----+-------+-------+
|    3|  박트롬|    여|    3|LG Cyon|1800000|
|    2|김싸이언|    남|    2|LG DIOS|2000000|
+-----+--------+------+-----+-------+-------+



DataFrame[u_uid: bigint]

### 3.3 중복 컬럼명 해결방안 - 조인 직후 중복 컬럼을 제거합니다

In [11]:
up = u.join(p, u.id == p.id).drop(p.id)
up.show()
up.select("id")

+---+--------+------+-------+-------+
| id|    name|gender|   name| amount|
+---+--------+------+-------+-------+
|  3|  박트롬|    여|LG Cyon|1800000|
|  2|김싸이언|    남|LG DIOS|2000000|
+---+--------+------+-------+-------+



DataFrame[id: bigint]

### <font color=red>7. [고급]</font> 고객 정보 "data/tbl_user_id", 제품 정보 "data/tbl_purchase_id" CSV 파일을 읽고
#### 1. *고객(tbl_user) 테이블의 아이디도 id 이고 제품(tbl_purchase) 테이블의 아이디도 id 입니다*
#### 2. 가장 비싼 제품을 구매한 고객의 고객정보와 제품정보를 출력해 주세요
#### 3. 최종 출력되는 컬럼은 고객아이디(u_id), 고객이름(u_name), 상품이름(p_name), 상품가격(p_amount) 이렇게 4개 컬럼입니다
#### 4. 상품가격 (p_amount) 내림차순으로 정렬되어야 합니다

<details><summary>[실습7] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
left = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/tbl_purchase_id.csv")
)
# left.printSchema()
# left.show()

right = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("data/tbl_user_id.csv")
)
# right.printSchema()
# right.show()

u_left = left.withColumnRenamed("id", "u_id")
u_right = right.withColumnRenamed("id", "p_uid")
u_left.printSchema()
u_right.printSchema()

join_codition = u_left.u_id == u_right.p_uid
answer = u_left.join(u_right, join_codition, "inner").where("p_amount > 0").select("u_id", "u_name", "p_name", "p_amount")
answer.printSchema()
display(answer.orderBy(desc("p_amount")))
```

</details>


In [35]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


root
 |-- p_time: integer (nullable = true)
 |-- u_id: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)

root
 |-- p_uid: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)

root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)



u_id,u_name,p_name,p_amount
4,청소기,LG Computer,4500000
5,유코드제로,LG Gram,3500000
5,유코드제로,LG TV,2500000
1,정휘센,LG DIOS,2000000
1,정휘센,LG Gram,1800000
2,김싸이언,LG Cyon,1400000
3,박트롬,LG TV,1000000


## 참고자료

#### 1. [Spark Programming Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html)
#### 2. [PySpark SQL Modules Documentation](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html)
#### 3. <a href="https://spark.apache.org/docs/3.0.1/api/sql/" target="_blank">PySpark 3.0.1 Builtin Functions</a>
#### 4. [PySpark Search](https://spark.apache.org/docs/latest/api/python/search.html)
#### 5. [Pyspark Functions](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?#module-pyspark.sql.functions)